<a href="https://colab.research.google.com/github/janlucasdeinhard/pyfiles/blob/master/10_PROJECT_TIMESERIES/Forecast/CryptoTracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


The goal of this notebook is to develop a visualization of useful KPIs for a set of selected crypto currencies. The KPIs should be easy to track and understand, and allow to forecast the near-term earning potential if one had purchased this portfolio. 

In [306]:
# Default libraries
import numpy as np
from numpy import random as rnd
from matplotlib import pyplot as plt
import warnings,datetime,time,math,itertools,os,sys

# Visualization libraries
import pandas as pd
import plotly.express as px
from plotly import graph_objects as go

# Request libabries to query up-to-date list of cryptos
import json
from bs4 import BeautifulSoup
import requests 

# Base library to query historical prices for given crypto
from cryptocmd import CmcScraper

In [130]:
def historical_prices_to_pandas(cryptoname):
  try:
    from cryptocmd import CmcScraper
  except:
    return pd.DataFrame()
  scraper = CmcScraper(cryptoname)
  colnames,contents = scraper.get_data()
  return pd.DataFrame(contents,columns=colnames)

In [9]:
url = "https://www.cryptocompare.com/api/data/coinlist/"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

In [30]:
data = json.loads(soup.prettify())
data = data['Data']

In [106]:
mf = pd.DataFrame(data).transpose()
mf = mf.sort_values(by=['SortOrder'],ascending=True).reset_index(drop=True)

mf = mf[[
    'Name', 
    'Symbol',
    'CoinName', 
    'FullName', 
    'Description', 
    #'Id', 
    #'Url', 
    #'ImageUrl', 
    #'ContentCreatedOn', 
    #'AssetTokenStatus', 
    'Algorithm',
    'ProofType', 
    #'SortOrder', 
    'Sponsored', 
    #'Taxonomy', 
    'Rating'
]]

mf = pd.concat([mf.drop(columns=['Rating']),mf.Rating.apply(pd.Series).Weiss.apply(pd.Series)],axis=1)

ratings_mapping = {
    'A+':1,
    'A':2,
    'A-':3,
    'B+':4,
    'B':5,
    'B-':6,
    'C+':7,
    'C':8,
    'C-':9,
    'D+':10,
    'D':11,
    'D-':12,
    'E+':13,
    'E':14,
    'E-':15
}

mf['Rating'] = mf['Rating'].map(ratings_mapping)
mf['TechnologyAdoptionRating'] = mf['TechnologyAdoptionRating'].map(ratings_mapping)
mf['MarketPerformanceRating'] = mf['MarketPerformanceRating'].map(ratings_mapping)

In [111]:
top20 = mf.sort_values(by=['Rating','MarketPerformanceRating','TechnologyAdoptionRating'],ascending=True).head(20)

In [114]:
top20.head()

,Name,Symbol,CoinName,FullName,Description,Algorithm,ProofType,Sponsored,Rating,TechnologyAdoptionRating,MarketPerformanceRating
0,BTC,BTC,Bitcoin,Bitcoin (BTC),Bitcoin uses peer-to-peer technology to operat...,SHA-256,PoW,False,4.0,3.0,6.0
598,KNC,KNC,Kyber Network,Kyber Network (KNC),KyberNetwork is an on-chain protocol which all...,N/A,N/A,False,5.0,6.0,3.0
3647,RUNE,RUNE,Thorchain,Thorchain (RUNE),THORCHain is a decentralized liquidity protoco...,N/A,N/A,False,5.0,5.0,5.0
3119,MATIC,MATIC,Polygon,Polygon (MATIC),"Matic provides scalable, secure and instant tr...",N/A,N/A,False,5.0,4.0,7.0
4528,DOT,DOT,Polkadot,Polkadot (DOT),What is Polkadot?Polkadot is a blockchain netw...,N/A,N/A,False,5.0,5.0,7.0


In [195]:
pf = pd.DataFrame()

for cname in top20['Symbol'].tolist():
  cf = historical_prices_to_pandas(cname)
  cf['Date'] = pd.to_datetime(cf['Date'])
  cf['Symbol'] = cname
  pf = pd.concat([pf,cf],axis=0)
pf = pf.reset_index(drop=True).sort_values(by=['Symbol','Date'],ascending=True)

In [197]:
px.line(
    data_frame=pf[pf['Symbol']=='RUNE'],
    x='Date',
    y='Close'
)

In [241]:
pf['Symbol'].unique()

array(['AAVE', 'AXS', 'BAT', 'BTC', 'CRO', 'DOT', 'ENJ', 'FTT', 'KNC',
       'LEO', 'LINK', 'LRC', 'LTC', 'MATIC', 'MKR', 'RUNE', 'SAND', 'XLM',
       'XRP', 'ZEC'], dtype=object)

In [353]:
csymbol = 'SAND'

date_launched = pf[pf['Symbol']==csymbol]['Date'].min()

tf = pf[pf['Symbol']==csymbol].set_index('Date')['Close'].rolling(10,win_type='cosine').mean().diff()

first_occurrence_high_derivative = tf[tf>=tf.quantile(0.97)].index.min()
highest_variability = pf[pf['Symbol']==csymbol][['Date','Close']].set_index('Date').rolling(100).std().idxmax().Close



fig = go.Figure()

fig.add_trace(
    go.Scatter(
        y=pf[pf['Symbol']==csymbol][['Close']].values.flatten(),
        x=pf[pf['Symbol']==csymbol][['Date']].values.flatten()
    )
)

fig.add_annotation(
    dict(
        font=dict(color='red',size=15),
        x=first_occurrence_high_derivative,
        y=pf[(pf['Symbol']==csymbol)&(pf['Date']==first_occurrence_high_derivative)].Close.values[0],
        showarrow=True,
        text="First Potential Indicator",
        xanchor='right',
        xref="x",
        yref="y")
)

fig.add_annotation(
    dict(
        font=dict(color='orange',size=15),
        x=highest_variability,
        y=pf[(pf['Symbol']==csymbol)&(pf['Date']==highest_variability)].Close.values[0],
        showarrow=True,
        text="Highest variability",
        xanchor='right',
        xref="x",
        yref="y")
)

##Potential Strategy

Empirical observation: Buying at redi indicator flag, i.e. when rise is first observable, and selling at the next high after a point of great variability, inidcating the presence of the profit maximal splikes, seems like a safe strategy to maxmimize profits. Question: How to determine these points a-priori, i.e. estimate the rolling mean and differential quantiles?  